This codes are from a book **[Functional Programming Simplified]** written by *Alvin Alexander*

In [ ]:
def f(a:Int): Int = a*2
def g(a:Int): Int = a*3

In [ ]:
val x = g(f(100))
println(x)

If the output of f is (Int, String)

In [4]:
def f(a: Int): (Int, String) = {
    val result = a*2
    (result, s"\nf result: $result.")
}

def g(a: Int): (Int, String) = {
    val result = a*3
    (result, s"\ng result: $result.")
}

def h(a: Int): (Int, String) = {
    val result = a*4
    (result, s"\nh result: $result.")
}

defined function f
defined function g
defined function h

In [ ]:
val (fInt, fString) = f(100)
val (gInt, gString) = g(fInt)

In [ ]:
val debug = fString + "" + gString
println(s"result: $gInt, debug: $debug")

While this approach works for this simple case, imagine what your code will look like when you need to string many more functions together. That would be an awful lot of manually written (and error-prone) code. We can do better.

## Bind Function

In [ ]:
def bind(fun: (Int) => (Int, String),
         tup: Tuple2[Int, String]): (Int, String) = 
{
    val (intResult, stringResult) = fun(tup._1)
    (intResult, tup._2 + stringResult)
}

In [ ]:
val fResult = f(100)
val gResult = bind(g, fResult)
val hResult = bind(h, gResult)

In [ ]:
println(s"result: ${hResult._1}, debug: ${hResult._2}")

If there’s anything bad to say about bind, it’s that it looks like it’s dying to be used in a for expression, but because bind doesn’t have methods like map and flatMap, it won’t work that way.

## Using a “Wrapper” Class in a for Expression

In [ ]:
class Wrapper[Int](value:Int) {
    def map(f: Int => Int): Wrapper[Int] = {
        val newInt = f(value)
        new Wrapper(newInt)
    }
    def flatMap(f: Int => Wrapper[Int]): Wrapper[Int] = f(value)
    
    override def toString = value.toString
}

In [ ]:
val result: Wrapper[Int] = for {
    a <- new Wrapper(1)
    b <- new Wrapper(2)
    c <- new Wrapper(3)
} yield a+b+c

## Generic Code

In [ ]:
class Wrapper[A](value: A) {
    def map[B](f: A => B): Wrapper[B] = {
        val newInt = f(value)
        new Wrapper(newInt)
    }
    def flatMap[B](f: A => Wrapper[B]): Wrapper[B] = f(value)
    
    override def toString = value.toString
}

In [ ]:
val intResult: Wrapper[Int] = for {
    a <- new Wrapper(1)
    b <- new Wrapper(2)
    c <- new Wrapper(3)
} yield a+b+c

In [ ]:
val stringResult: Wrapper[String] = for {
    a <- new Wrapper("a")
    b <- new Wrapper("b")
    c <- new Wrapper("c")
} yield a+b+c

## Creating an apply method in a companion object

In [1]:
// 1) make the Wrapper class constructor private
class Wrapper[A] private (value: A) {
    def map[B](f: A => B): Wrapper[B] = {
        val newValue = f(value)
        new Wrapper(newValue)
    }
    def flatMap[B](f: A => Wrapper[B]): Wrapper[B] = {
        val newValue = f(value)
        newValue
    }
    override def toString = value.toString
}

// 2)Create an object named Wrapper in the same file as the Wrapper class
object Wrapper {
    def apply[A](value: A): Wrapper[A] = new Wrapper(value)
}

defined class Wrapper
defined object Wrapper

In [2]:
// 3) Create an apply method in the companion object with the appropriate signature
val intResult = for {
    a <- Wrapper(1)
    b <- Wrapper(2)
    c <- Wrapper(3)
} yield a + b + c

intResult: Wrapper[Int] = 6

## Using bind in for expression

In [5]:
case class Debuggable(value: Int, message: String) {
    def map(f: Int => Int): Debuggable = {
        val nextValue = f(value)
        Debuggable(nextValue, message)
    }
    
    def flatMap(f: Int => Debuggable): Debuggable = {
        val nextValue: Debuggable = f(value)
        Debuggable(nextValue.value, message + "\n" + nextValue.message)
    }
}

defined class Debuggable

In [7]:
def f(a: Int): Debuggable = {
    val result = a * 2
    val message = s"f: a ($a) * 2 = $result."
    Debuggable(result, message)
}

def g(a: Int): Debuggable = {
    val result = a * 3
    val message = s"g: a ($a) * 3 = $result."
    Debuggable(result, message)
}

def h(a: Int): Debuggable = {
    val result = a * 4
    val message = s"h: a ($a) * 4 = $result."
    Debuggable(result, message)
}

defined function f
defined function g
defined function h

In [8]:
val finalResult = for {
    fResult <- f(100)
    gResult <- g(fResult)
    hResult <- h(gResult)
} yield hResult

finalResult: Debuggable = Debuggable(
  2400,
  """
f: a (100) * 2 = 200.  
g: a (200) * 3 = 600.  
h: a (600) * 4 = 2400.
  """
)